In [14]:

import pandas as pd


eventos = pd.read_csv('logs_entrenamiento.csv')

notas = pd.read_csv('notas.csv')

column_names_df = notas.columns


column_names_eventos = eventos.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'eventos':", column_names_eventos)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

notas = notas.replace('Not Attempted', 0)

print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

In [15]:
#ACÁ SE TOMA INFO DE LOG_TRAIN PARA SUPLIR LA DATA DE NOTAS


# Convert the 'time_column' to a datetime object
eventos['time'] = pd.to_datetime(eventos['time'])

# Create new columns for month, day, and time
eventos['month'] = eventos['time'].dt.month
eventos['day'] = eventos['time'].dt.day
eventos['hora'] = eventos['time'].dt.strftime('%H:%M:%S')




#----------------------------------

# DURACION DE SESIONES (una sesion es un día con registro de ingreso)

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

#
grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
eventos = eventos.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')



#----------------------------------

#ESTADISTICOS DE DUACION DE SESIONES

# Group the DataFrame by 'usuarioname' and calculate the average and standard deviation of 'time_difference'
grouped = eventos.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'average_time_difference', 'std_time_difference']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'average_time_difference', 'std_time_difference']].drop_duplicates(), on='username', how='left')



#----------------------------------

#DURACION EN DÍAS DE SU USO DE EOL

# Extract the date part
eventos['date'] = eventos['time'].dt.date

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username'])['date'].agg(['min', 'max']).reset_index()

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')




In [16]:

#----------------------------------

#FRECUENCIA DE SESIONES POR MES


distinct_months = eventos['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': eventos['username']}
for month in distinct_months:
    result_data[month] = eventos['username'].map(
        eventos[eventos['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={'8': 'Num Sesiones de Agosto', '9': 'Num Sesiones de Septiembre', '10': 'Num Sesiones de Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
3                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
4                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
...            ...                                                ...    ...   
536764         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536765         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536766         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536767         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536768         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

In [17]:
# Get unique values from a specific column
unique_values = eventos['grouped_event_type'].unique()
print(unique_values)

['/courses/(course-code)/course/'
 'edx.bi.course.upgrade.sidebarupsell.displayed'
 '/courses/(course-code)/progress' 'edx.ui.lms.link_clicked'
 '/courses/(course-code)/(uuid-course)/'
 '/courses/(course-code)/course_wiki'
 '/courses/(course-code)/wiki/(course-code)/'
 '/courses/(course-code)/discussion/(discussion-id)/'
 '/courses/(course-code)/jump_to/(block-code)@vertical@(uuid)'
 '/courses/(course-code)/courseware/(uuid1)/(uuid2)/(tab-n)'
 '/courses/(course-code)/xblock/block-v1:(block-code)@html+block@(uuid)/handler/publish_completion'
 'edx.ui.lms.sequence.next_selected'
 '/courses/(course-code)/courseware/(uuid-code1)/(uuid-code2)/(some-path)'
 'load_video'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/get_completion'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/goto_position'
 'seq_goto' 'play_video' 'pause_video' 'seek_video' 'page_close'
 '/courses/(course-code)/xb

In [18]:


# List of event types to create columns for
event_types = ['page_close', 'problem_graded', 'problem_check', 'problem_show', 'seg_prev',
               'seg_next', 'seg_goto', 'load_video', 'play_video', 'pause_video', 'speed_change_video']

# Iterate through the event types and create columns with boolean values
for event_type in event_types:
    eventos[event_type] = (eventos['grouped_event_type'] == event_type).astype(int)

grouped_eventos = eventos.groupby(['username', 'month', 'day'])[event_types].sum().reset_index()



# Group the sub_df DataFrame by 'username' and calculate the mean and standard deviation for each event type
agg_df = grouped_eventos.groupby('username').agg({
    'page_close': ['mean', 'std'],
    'problem_graded': ['mean', 'std'],
    'problem_check': ['mean', 'std'],
    'problem_show': ['mean', 'std'],
    'seg_prev': ['mean', 'std'],
    'seg_next': ['mean', 'std'],
    'seg_goto': ['mean', 'std'],
    'load_video': ['mean', 'std'],
    'play_video': ['mean', 'std'],
    'pause_video': ['mean', 'std'],
    'speed_change_video': ['mean', 'std']
}).reset_index()

print(agg_df)

# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(agg_df, on='username', how='left')

# Display the resulting 'notas' DataFrame
print(notas)


                                              username page_close            \
                                                             mean       std   
0    00c20ed7115c435d81db24d0e68e9bd610de97dda63586...   0.000000  0.000000   
1    00c245b6f4beeba47da73c2031bd7d6a46afafa9e36044...   0.000000  0.000000   
2    02f92a5a8536d16db362937ea2dd48d902417b40136cb7...   3.857143  3.899845   
3    02fdd011a6084ac4f0b1b47a08bec7c228e9ee20017ed5...   0.000000  0.000000   
4    03800e3c25b0e755d8a3c735c5721a4d643071bfbe36cb...   0.714286  1.496026   
..                                                 ...        ...       ...   
423  fd8e5ea29345e6f16faefdc6b702bce65cdad7ee050c87...   0.000000  0.000000   
424  fe345db7771b1002b45e386a7a3bdc24e0d4d09261c7cc...   0.000000  0.000000   
425  fe6fc7f1c7d4beaa8911b553ddee756d9f89d5bfc0c67b...   0.000000  0.000000   
426  fed6397382310b49ef5b4aa593b34dd47a665fe4e20b6a...   4.222222  3.073181   
427  ff901599173460ccfbbf0fc45c3a78969795e0b3111d64.

/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  indexer = self._engine.get_indexer(target._get_engine_target())


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
3                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
4                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
...            ...                                                ...    ...   
536764         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536765         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536766         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536767         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536768         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:4574: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self._engine


In [19]:
notas.to_csv('data_para_entrenar.csv')

#PENDIENTE

- encontrar forma de incluir las variables de event type (conocer los distintos tipos de event type y sacar la frecuencia de cada uno en las sesiones y a partir de esto sacar estadisticos, ej en promedio cuantas veces pone pausa, a que vwlocidad ve ek video, repite mucho los mismos videos etc)
-quizas los estadísticos de la duracion de las sesiones podría ser especifico a cada mes
